In [1]:
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizerFast, DataCollatorWithPadding, pipeline
from datasets import load_metric, Dataset
import numpy as np
import torch

/opt/homebrew/Caskroom/miniconda/base/envs/pytorch_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
snips_file=open("Data/Snips Train Data.txt")
snips_rows=snips_file.readlines()
snips_rows[:20]

['listen O\n',
 'to O\n',
 'westbam B-artist\n',
 'alumb O\n',
 'allergic B-album\n',
 'on O\n',
 'google B-service\n',
 'music I-service\n',
 'PlayMusic\n',
 '\n',
 'add O\n',
 'step B-entity_name\n',
 'to I-entity_name\n',
 'me I-entity_name\n',
 'to O\n',
 'the O\n',
 '50 B-playlist\n',
 'clásicos I-playlist\n',
 'playlist O\n',
 'AddToPlaylist\n']

In [3]:
utterances = []
tokenized_utterances = []
labels_for_tokens = []
sequence_labels = []
utterance, tokenized_utterance, label_for_utterances = '', [], []

for snip_row in snips_rows:
    if len(snip_row) == 2:  # Skip rows with insufficient data
        continue
    if ' ' not in snip_row.strip():  # Sequence label
        sequence_label = snip_row.strip()
        if sequence_label:  # Only add non-empty labels
            sequence_labels.append(sequence_label)
        if utterance.strip():  # Avoid appending empty utterances
            utterances.append(utterance.strip())
            tokenized_utterances.append(tokenized_utterance)
            labels_for_tokens.append(label_for_utterances)
        # Reset for next sequence
        utterance = ''
        tokenized_utterance = []
        label_for_utterances = []
        continue
    token, token_label = snip_row.split(' ')
    token_label = token_label.strip()
    utterance += f'{token} '
    tokenized_utterance.append(token)
    label_for_utterances.append(token_label)


In [4]:
len(labels_for_tokens) , len(tokenized_utterances), len(utterances), len(sequence_labels)

(13084, 13084, 13084, 13084)

In [5]:
print(tokenized_utterances[0])
print(labels_for_tokens[0])
print(utterances[0])
print(sequence_labels[0])

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']
['O', 'O', 'B-artist', 'O', 'B-album', 'O', 'B-service', 'I-service']
listen to westbam alumb allergic on google music
PlayMusic


In [6]:
unique_sequence_labels=list(set(sequence_labels))
sequence_labels=[unique_sequence_labels.index(l) for l in sequence_labels]
print(len(unique_sequence_labels))
print(unique_sequence_labels)

7
['GetWeather', 'BookRestaurant', 'SearchCreativeWork', 'PlayMusic', 'SearchScreeningEvent', 'AddToPlaylist', 'RateBook']


In [7]:
from functools import reduce

unique_token_labels=list(set(reduce(lambda x, y:x+y,labels_for_tokens)))
labels_for_tokens=[[unique_token_labels.index(_) for _ in l]for l in labels_for_tokens]
print(len(unique_token_labels))

72


In [8]:
print(tokenized_utterances[0])
print(labels_for_tokens[0])
print(utterances[0])
print(sequence_labels[0])

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']
[28, 28, 55, 28, 44, 28, 41, 36]
listen to westbam alumb allergic on google music
3


In [9]:
snips_dataset=Dataset.from_dict(dict(utterance=utterances,label=sequence_labels,tokens=tokenized_utterances,token_label=labels_for_tokens))
snips_dataset=snips_dataset.train_test_split(0.2)

In [10]:
print(snips_dataset['train'][0])

{'utterance': 'restaurant in south sudan for 6', 'label': 1, 'tokens': ['restaurant', 'in', 'south', 'sudan', 'for', '6'], 'token_label': [27, 28, 53, 59, 28, 29]}


In [11]:
tokenizer=DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [12]:
def preprocess_function(examples):
    return tokenizer(examples["utterance"],truncation=True)

In [13]:
seq_clf_tokenized_snips = snips_dataset.map(preprocess_function,batched=True)

100%|██████████| 3/3 [00:00<00:00, 10.51ba/s]


In [14]:
seq_clf_tokenized_snips['train'][0]

{'utterance': 'restaurant in south sudan for 6',
 'label': 1,
 'tokens': ['restaurant', 'in', 'south', 'sudan', 'for', '6'],
 'token_label': [27, 28, 53, 59, 28, 29],
 'input_ids': [101, 4825, 1999, 2148, 10411, 2005, 1020, 102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [15]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [16]:
seq_clf_model=DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=len(unique_sequence_labels))
seq_clf_model.config.id2label={i:l for i,l in enumerate(unique_sequence_labels)}

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
seq_clf_model.config.id2label[0]

'GetWeather'

In [18]:
metric = load_metric("accuracy")
def compute_matrics(eval_pred):
    logists,labels=eval_pred
    predictions=np.argmax(logists,axis=-1)
    return metric.compute(predictions=predictions,references=labels)

In [19]:
epochs = 2
warmup_steps = len(seq_clf_tokenized_snips['train']) // 5
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

training_arguments = TrainingArguments(
    output_dir='./snips_clf/results',
    num_train_epochs=epochs,
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    load_best_model_at_end=True,
    warmup_steps=warmup_steps,
    weight_decay=0.05,
    logging_steps=1,
    log_level="info",
    eval_strategy='epoch',
    save_strategy='epoch',
    no_cuda=True
)

seq_clf_model.to(device)
trainer=Trainer(model=seq_clf_model,args=training_arguments,train_dataset=seq_clf_tokenized_snips['train'],eval_dataset=seq_clf_tokenized_snips['test'],compute_metrics=compute_matrics,data_collator=data_collator)

/opt/homebrew/Caskroom/miniconda/base/envs/pytorch_env/lib/python3.8/site-packages/transformers/training_args.py:1583: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(


In [20]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_label, tokens. If utterance, token_label, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
100%|██████████| 82/82 [00:27<00:00,  3.00it/s]


{'eval_loss': 1.946377158164978,
 'eval_model_preparation_time': 0.0011,
 'eval_accuracy': 0.13068398930072603,
 'eval_runtime': 27.6335,
 'eval_samples_per_second': 94.704,
 'eval_steps_per_second': 2.967}

In [21]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_label, tokens. If utterance, token_label, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10,467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656
  Number of trainable parameters = 66,958,855
  0%|          | 1/656 [00:00<07:45,  1.41it/s]

{'loss': 1.9455, 'grad_norm': 1.4432977437973022, 'learning_rate': 2.3889154323936934e-08, 'epoch': 0.0}


  0%|          | 2/656 [00:01<08:27,  1.29it/s]

{'loss': 1.9424, 'grad_norm': 1.0646190643310547, 'learning_rate': 4.777830864787387e-08, 'epoch': 0.01}


  0%|          | 3/656 [00:02<07:48,  1.39it/s]

...

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_label, tokens. If utterance, token_label, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
                                                 
100%|██████████| 656/656 [09:40<00:00,  1.52it/s]Saving model checkpoint to ./snips_clf/results/checkpoint-656
Configuration saved in ./snips_clf/results/checkpoint-656/config.json
Model weights saved in ./snips_clf/results/checkpoint-656/model.safetensors


{'eval_loss': 0.06383088231086731, 'eval_model_preparation_time': 0.0011, 'eval_accuracy': 0.9870080244554834, 'eval_runtime': 20.836, 'eval_samples_per_second': 125.6, 'eval_steps_per_second': 3.935, 'epoch': 2.0}




Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./snips_clf/results/checkpoint-656 (score: 0.06383088231086731).
100%|██████████| 656/656 [09:41<00:00,  1.13it/s]

{'train_runtime': 581.0366, 'train_samples_per_second': 36.029, 'train_steps_per_second': 1.129, 'train_loss': 0.6983654683004361, 'epoch': 2.0}


TrainOutput(global_step=656, training_loss=0.6983654683004361, metrics={'train_runtime': 581.0366, 'train_samples_per_second': 36.029, 'train_steps_per_second': 1.129, 'total_flos': 116893238379912.0, 'train_loss': 0.6983654683004361, 'epoch': 2.0})

In [22]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_label, tokens. If utterance, token_label, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
100%|██████████| 82/82 [00:20<00:00,  3.97it/s]


{'eval_loss': 0.06383088231086731,
 'eval_model_preparation_time': 0.0011,
 'eval_accuracy': 0.9870080244554834,
 'eval_runtime': 20.8378,
 'eval_samples_per_second': 125.589,
 'eval_steps_per_second': 3.935,
 'epoch': 2.0}

In [23]:
pipe=pipeline('text-classification', seq_clf_model,tokenizer=tokenizer)
pipe('Add Two Coins by dispatch to my road trip playlist')

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


[{'label': 'AddToPlaylist', 'score': 0.990362823009491}]

In [24]:
trainer.save_model()

Saving model checkpoint to ./snips_clf/results
Configuration saved in ./snips_clf/results/config.json
Model weights saved in ./snips_clf/results/model.safetensors


In [25]:
pipe=pipeline('text-classification', './snips_clf/results/',tokenizer=tokenizer)
pipe('Add Two Coins by dispatch to my road trip playlist')

loading configuration file ./snips_clf/results/config.json
Model config DistilBertConfig {
  "_name_or_path": "./snips_clf/results/",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "GetWeather",
    "1": "BookRestaurant",
    "2": "SearchCreativeWork",
    "3": "PlayMusic",
    "4": "SearchScreeningEvent",
    "5": "AddToPlaylist",
    "6": "RateBook"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transfo

[{'label': 'AddToPlaylist', 'score': 0.990362823009491}]

In [36]:
frozen_sequence_clf_model=DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=len(unique_sequence_labels))
frozen_sequence_clf_model.config.id2label={i:l for i,l in enumerate(unique_sequence_labels)}

loading configuration file config.json from cache at /Users/vishalsankarram/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/12040accade4e8a0f71eabdb258fecc2e7e948be/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.46.2",
  "vocab_size": 

In [37]:
for param in frozen_sequence_clf_model.distilbert.parameters():
    param.requires_grad = False
for param in frozen_sequence_clf_model.classifier.parameters():
    param.requires_grad = True

In [38]:
epochs = 2
warmup_steps = len(seq_clf_tokenized_snips['train']) // 5
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

training_arguments = TrainingArguments(
    output_dir='./snips_clf/results',
    num_train_epochs=epochs,
    per_device_eval_batch_size=32,
    per_device_train_batch_size=32,
    load_best_model_at_end=True,
    warmup_steps=warmup_steps,
    weight_decay=0.05,
    logging_steps=1,
    log_level="info",
    eval_strategy='epoch',
    save_strategy='epoch',
    no_cuda=True
)

frozen_sequence_clf_model.to(device)
trainer=Trainer(model=frozen_sequence_clf_model,args=training_arguments,train_dataset=seq_clf_tokenized_snips['train'],eval_dataset=seq_clf_tokenized_snips['test'],compute_metrics=compute_matrics,data_collator=data_collator)

/opt/homebrew/Caskroom/miniconda/base/envs/pytorch_env/lib/python3.8/site-packages/transformers/training_args.py:1583: FutureWarning: using `no_cuda` is deprecated and will be removed in version 5.0 of 🤗 Transformers. Use `use_cpu` instead
  warnings.warn(
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [34]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_label, tokens. If utterance, token_label, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
100%|██████████| 82/82 [00:16<00:00,  5.11it/s]


{'eval_loss': 1.9490116834640503,
 'eval_model_preparation_time': 0.0018,
 'eval_accuracy': 0.14902560183416125,
 'eval_runtime': 16.2085,
 'eval_samples_per_second': 161.458,
 'eval_steps_per_second': 5.059}

In [39]:
trainer.train()

  0%|          | 0/656 [06:53<?, ?it/s]
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_label, tokens. If utterance, token_label, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 10,467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656
  Number of trainable parameters = 595,975
                                               
  0%|          | 1/656 [00:00<02:17,  4.76it/s]

{'loss': 1.964, 'grad_norm': 1.3711175918579102, 'learning_rate': 2.3889154323936934e-08, 'epoch': 0.0}


                                               
  0%|          | 2/656 [00:00<02:50,  3.84it/s]

{'loss': 1.9509, 'grad_norm': 0.9557554721832275, 'learning_rate': 4.777830864787387e-08, 'epoch': 0.01}


                                               
  0%|          | 3/656 [00:00<02:36,  4.16it/s]

...

                                                 
                                                 
100%|██████████| 656/656 [03:23<00:00,  3.48it/s]Saving model checkpoint to ./snips_clf/results/checkpoint-656
Configuration saved in ./snips_clf/results/checkpoint-656/config.json
Model weights saved in ./snips_clf/results/checkpoint-656/model.safetensors
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_label, tokens. If utterance, token_label, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32


{'loss': 1.6064, 'grad_norm': 1.3096256256103516, 'learning_rate': 1.564739608217869e-05, 'epoch': 2.0}
{'loss': 1.499, 'grad_norm': 3.8476948738098145, 'learning_rate': 1.5671285236502626e-05, 'epoch': 2.0}




















































































                                                 
                                    

100%|██████████| 656/656 [03:47<00:00,  3.48it/s]

Saving model checkpoint to ./snips_clf/results/checkpoint-656
Configuration saved in ./snips_clf/results/checkpoint-656/config.json
Model weights saved in ./snips_clf/results/checkpoint-656/model.safetensors


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./snips_clf/results/checkpoint-656 (score: 1.5848826169967651).
                                                 
100%|██████████| 656/656 [03:47<00:00,  2.89it/s]

{'eval_loss': 1.5848826169967651, 'eval_accuracy': 0.7875429881543753, 'eval_runtime': 23.749, 'eval_samples_per_second': 110.194, 'eval_steps_per_second': 3.453, 'epoch': 2.0}
{'train_runtime': 227.1811, 'train_samples_per_second': 92.147, 'train_steps_per_second': 2.888, 'train_loss': 1.8466433021353512, 'epoch': 2.0}


TrainOutput(global_step=656, training_loss=1.8466433021353512, metrics={'train_runtime': 227.1811, 'train_samples_per_second': 92.147, 'train_steps_per_second': 2.888, 'total_flos': 116893238379912.0, 'train_loss': 1.8466433021353512, 'epoch': 2.0})

In [40]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: utterance, token_label, tokens. If utterance, token_label, tokens are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.

***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
100%|██████████| 82/82 [00:15<00:00,  5.28it/s]


{'eval_loss': 1.5848826169967651,
 'eval_accuracy': 0.7875429881543753,
 'eval_runtime': 15.7042,
 'eval_samples_per_second': 166.643,
 'eval_steps_per_second': 5.222,
 'epoch': 2.0}